In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
PATH = "/content/drive/MyDrive/playground-series-s5e10"

In [12]:
import polars as pl
from polars import selectors as cs
import lightgbm as lgb

In [26]:
from lightgbm import early_stopping
import mlflow
import mlflow.lightgbm
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.metrics import roc_auc_score

In [4]:
!ls

sample_submission.csv  test.csv  train.csv


In [1]:
PATH = "playground-series-s5e10"

In [2]:
%cd $PATH

/Users/kavya/accident-risk-prediction/playground-series-s5e10


/Users/kavya/accident-risk-prediction/venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
data = pl.read_csv("train.csv")

In [6]:
data

id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
i64,str,i64,f64,i64,str,str,bool,bool,str,bool,bool,i64,f64
0,"""urban""",2,0.06,35,"""daylight""","""rainy""",false,true,"""afternoon""",false,true,1,0.13
1,"""urban""",4,0.99,35,"""daylight""","""clear""",true,false,"""evening""",true,true,0,0.35
2,"""rural""",4,0.63,70,"""dim""","""clear""",false,true,"""morning""",true,false,2,0.3
3,"""highway""",4,0.07,35,"""dim""","""rainy""",true,true,"""morning""",false,false,1,0.21
4,"""rural""",1,0.58,60,"""daylight""","""foggy""",false,false,"""evening""",true,false,1,0.56
…,…,…,…,…,…,…,…,…,…,…,…,…,…
517749,"""highway""",4,0.1,70,"""daylight""","""foggy""",true,true,"""afternoon""",false,false,2,0.32
517750,"""rural""",4,0.47,35,"""daylight""","""rainy""",true,true,"""morning""",false,false,1,0.26
517751,"""urban""",4,0.62,25,"""daylight""","""foggy""",false,false,"""afternoon""",false,true,0,0.19


In [7]:
train = pl.DataFrame(data)


In [8]:
print(train.null_count())

shape: (1, 14)
┌─────┬───────────┬───────────┬───────────┬───┬─────────┬──────────────┬─────────────┬─────────────┐
│ id  ┆ road_type ┆ num_lanes ┆ curvature ┆ … ┆ holiday ┆ school_seaso ┆ num_reporte ┆ accident_ri │
│ --- ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---     ┆ n            ┆ d_accidents ┆ sk          │
│ u32 ┆ u32       ┆ u32       ┆ u32       ┆   ┆ u32     ┆ ---          ┆ ---         ┆ ---         │
│     ┆           ┆           ┆           ┆   ┆         ┆ u32          ┆ u32         ┆ u32         │
╞═════╪═══════════╪═══════════╪═══════════╪═══╪═════════╪══════════════╪═════════════╪═════════════╡
│ 0   ┆ 0         ┆ 0         ┆ 0         ┆ … ┆ 0       ┆ 0            ┆ 0           ┆ 0           │
└─────┴───────────┴───────────┴───────────┴───┴─────────┴──────────────┴─────────────┴─────────────┘


In [11]:
string_cols = train.select(cs.string()).columns
print(string_cols)

['road_type', 'lighting', 'weather', 'time_of_day']


In [13]:
train

id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
i64,str,i64,f64,i64,str,str,bool,bool,str,bool,bool,i64,f64
0,"""urban""",2,0.06,35,"""daylight""","""rainy""",false,true,"""afternoon""",false,true,1,0.13
1,"""urban""",4,0.99,35,"""daylight""","""clear""",true,false,"""evening""",true,true,0,0.35
2,"""rural""",4,0.63,70,"""dim""","""clear""",false,true,"""morning""",true,false,2,0.3
3,"""highway""",4,0.07,35,"""dim""","""rainy""",true,true,"""morning""",false,false,1,0.21
4,"""rural""",1,0.58,60,"""daylight""","""foggy""",false,false,"""evening""",true,false,1,0.56
…,…,…,…,…,…,…,…,…,…,…,…,…,…
517749,"""highway""",4,0.1,70,"""daylight""","""foggy""",true,true,"""afternoon""",false,false,2,0.32
517750,"""rural""",4,0.47,35,"""daylight""","""rainy""",true,true,"""morning""",false,false,1,0.26
517751,"""urban""",4,0.62,25,"""daylight""","""foggy""",false,false,"""afternoon""",false,true,0,0.19


(517754, 13)
(517754, 1)


In [14]:
read_test_data = pl.read_csv("test.csv")
test = pl.DataFrame(read_test_data)
print(test)

shape: (172_585, 13)
┌────────┬───────────┬───────────┬───────────┬───┬─────────────┬─────────┬────────────┬────────────┐
│ id     ┆ road_type ┆ num_lanes ┆ curvature ┆ … ┆ time_of_day ┆ holiday ┆ school_sea ┆ num_report │
│ ---    ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---         ┆ ---     ┆ son        ┆ ed_acciden │
│ i64    ┆ str       ┆ i64       ┆ f64       ┆   ┆ str         ┆ bool    ┆ ---        ┆ ts         │
│        ┆           ┆           ┆           ┆   ┆             ┆         ┆ bool       ┆ ---        │
│        ┆           ┆           ┆           ┆   ┆             ┆         ┆            ┆ i64        │
╞════════╪═══════════╪═══════════╪═══════════╪═══╪═════════════╪═════════╪════════════╪════════════╡
│ 517754 ┆ highway   ┆ 2         ┆ 0.34      ┆ … ┆ afternoon   ┆ true    ┆ true       ┆ 1          │
│ 517755 ┆ urban     ┆ 3         ┆ 0.04      ┆ … ┆ afternoon   ┆ true    ┆ false      ┆ 0          │
│ 517756 ┆ urban     ┆ 2         ┆ 0.59      ┆ … ┆ afternoon   ┆ true 

In [32]:
# result_string_cols = train.select(cs.string())
# result_bool_cols = train.select(cs.boolean())
# train = train.with_columns([
#     pl.col(c).cast(pl.Categorical).to_physical().alias(c)
#     for c in train.columns if train[c].dtype == pl.Utf8
# ])

# bool_cols = train.select(
#     cs.boolean().cast(pl.Int8)
# )

# test = test.select(
#     cs.boolean().cast(pl.Int8)
# )

In [15]:
train = train.with_columns(
    cs.string().cast(pl.Categorical).to_physical(),
    cs.boolean().cast(pl.Int8)
)

test = test.with_columns(
    cs.string().cast(pl.Categorical).to_physical(),
    cs.boolean().cast(pl.Int8)
)

In [16]:
# string_cols = train.select(cs.string()).columns

# for col in string_cols:
#     train = train.with_columns(pl.col(col).cast(pl.Categorical).to_physical())
#     test = test.with_columns(pl.col(col).cast(pl.Categorical).to_physical())



print(train.head())
print(test.head())

shape: (5, 14)
┌─────┬───────────┬───────────┬───────────┬───┬─────────┬──────────────┬─────────────┬─────────────┐
│ id  ┆ road_type ┆ num_lanes ┆ curvature ┆ … ┆ holiday ┆ school_seaso ┆ num_reporte ┆ accident_ri │
│ --- ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---     ┆ n            ┆ d_accidents ┆ sk          │
│ i64 ┆ u32       ┆ i64       ┆ f64       ┆   ┆ i8      ┆ ---          ┆ ---         ┆ ---         │
│     ┆           ┆           ┆           ┆   ┆         ┆ i8           ┆ i64         ┆ f64         │
╞═════╪═══════════╪═══════════╪═══════════╪═══╪═════════╪══════════════╪═════════════╪═════════════╡
│ 0   ┆ 1         ┆ 2         ┆ 0.06      ┆ … ┆ 0       ┆ 1            ┆ 1           ┆ 0.13        │
│ 1   ┆ 1         ┆ 4         ┆ 0.99      ┆ … ┆ 1       ┆ 1            ┆ 0           ┆ 0.35        │
│ 2   ┆ 5         ┆ 4         ┆ 0.63      ┆ … ┆ 1       ┆ 0            ┆ 2           ┆ 0.3         │
│ 3   ┆ 0         ┆ 4         ┆ 0.07      ┆ … ┆ 0       ┆ 0            ┆ 1  

In [37]:
# bool_cols

road_signs_present,public_road,holiday,school_season
i8,i8,i8,i8
0,1,0,1
1,0,1,1
0,1,1,0
1,1,0,0
0,0,1,0
…,…,…,…
1,1,0,0
1,1,0,0
0,0,0,1


In [36]:
# for col in bool_cols.columns:
#     train = train.with_columns(pl.col(col).cast(pl.Int8).alias(col))
#     test  = test.with_columns(pl.col(col).cast(pl.Int8).alias(col))

In [17]:
X = train.drop('accident_risk')
y = train.select("accident_risk")
print(X.shape)
print(y.shape)

(517754, 13)
(517754, 1)


In [21]:
from sklearn.model_selection import train_test_split

# Split data (e.g. 80% train, 20% validation)
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

categorical_cols_indices = [X.columns.index(col) for col in string_cols]
# Convert to LightGBM Dataset format
train_data = lgb.Dataset(X_train, label=y_train.to_numpy().flatten(), categorical_feature=categorical_cols_indices)
valid_data = lgb.Dataset(X_valid, label=y_valid.to_numpy().flatten(), reference=train_data)

# Define parameters
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'verbose': -1
}


In [70]:

# train_data = lgb.Dataset(X_train, y_train.to_numpy().flatten(), )
# Train the model
    bst = lgb.train(
        params,
        train_data,
        num_boost_round=100
        # valid_sets=[valid_data],
        # callbacks=[early_stopping(10)]  # 👈 optional but recommended
    )

In [ ]:
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'verbose': -1
}



In [31]:
print(X_valid)

shape: (103_551, 13)
┌────────┬───────────┬───────────┬───────────┬───┬─────────────┬─────────┬────────────┬────────────┐
│ id     ┆ road_type ┆ num_lanes ┆ curvature ┆ … ┆ time_of_day ┆ holiday ┆ school_sea ┆ num_report │
│ ---    ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---         ┆ ---     ┆ son        ┆ ed_acciden │
│ i64    ┆ u32       ┆ i64       ┆ f64       ┆   ┆ u32         ┆ i8      ┆ ---        ┆ ts         │
│        ┆           ┆           ┆           ┆   ┆             ┆         ┆ i8         ┆ ---        │
│        ┆           ┆           ┆           ┆   ┆             ┆         ┆            ┆ i64        │
╞════════╪═══════════╪═══════════╪═══════════╪═══╪═════════════╪═════════╪════════════╪════════════╡
│ 50309  ┆ 0         ┆ 1         ┆ 0.09      ┆ … ┆ 6           ┆ 1       ┆ 0          ┆ 2          │
│ 95219  ┆ 1         ┆ 4         ┆ 0.14      ┆ … ┆ 8           ┆ 1       ┆ 0          ┆ 0          │
│ 197653 ┆ 0         ┆ 2         ┆ 0.51      ┆ … ┆ 7           ┆ 0    

In [30]:
print(X_test)

shape: (103_551, 13)
┌────────┬───────────┬───────────┬───────────┬───┬─────────────┬─────────┬────────────┬────────────┐
│ id     ┆ road_type ┆ num_lanes ┆ curvature ┆ … ┆ time_of_day ┆ holiday ┆ school_sea ┆ num_report │
│ ---    ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---         ┆ ---     ┆ son        ┆ ed_acciden │
│ i64    ┆ u32       ┆ i64       ┆ f64       ┆   ┆ u32         ┆ i8      ┆ ---        ┆ ts         │
│        ┆           ┆           ┆           ┆   ┆             ┆         ┆ i8         ┆ ---        │
│        ┆           ┆           ┆           ┆   ┆             ┆         ┆            ┆ i64        │
╞════════╪═══════════╪═══════════╪═══════════╪═══╪═════════════╪═════════╪════════════╪════════════╡
│ 50309  ┆ 0         ┆ 1         ┆ 0.09      ┆ … ┆ 6           ┆ 1       ┆ 0          ┆ 2          │
│ 95219  ┆ 1         ┆ 4         ┆ 0.14      ┆ … ┆ 8           ┆ 1       ┆ 0          ┆ 0          │
│ 197653 ┆ 0         ┆ 2         ┆ 0.51      ┆ … ┆ 7           ┆ 0    

In [28]:
print(y_pred)

[0.1330367  0.33052479 0.25688754 ... 0.41195096 0.1397529  0.15766032]


In [33]:
with mlflow.start_run() as run:
    
    # Train model
    bst = lgb.train(params, train_data, num_boost_round=100, valid_sets=[valid_data], callbacks=[early_stopping(10)])    
    y_pred = bst.predict(X_valid)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    r2 = r2_score(y_valid, y_pred)
    
    mlflow.log_metric("val_rmse", float(rmse))
    mlflow.log_metric("val_r2", float(r2))
    
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")

Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 0.056546
RMSE: 0.0565
R²: 0.8842


/Users/kavya/accident-risk-prediction/venv/lib/python3.9/site-packages/lightgbm/basic.py:1238: UserWarning: Converting data to scipy sparse matrix.
  _log_warning("Converting data to scipy sparse matrix.")


In [84]:


# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# r2 = r2_score(y_test, y_pred)

# print(f"RMSE: {rmse:.4f}")
# print(f"R²: {r2:.4f}")

RMSE: 0.0565
R²: 0.8842
